In [1]:
from itertools import product
import json
import os

from tqdm.auto import tqdm
import pandas as pd

OUTPUT_DIR = "../output/a100"
LLM_OUTPUT_DIR = "../output/llm"

In [2]:
experiments = list()

for experiment_name in tqdm(os.listdir(OUTPUT_DIR)):
    if experiment_name == "output":
        continue

    experiment_path = os.path.join(OUTPUT_DIR, experiment_name)

    raw_params = experiment_name.split("_")

    if "extra" in raw_params[1]:
        raw_params[0] = "processed_extra"
        raw_params.remove("extra")
    elif "processed" in experiment_path and "COVID19" in experiment_path and "v5" in experiment_path:
        continue

    params = {
        "data": raw_params[0],
        "dataset": raw_params[1].replace("-", "."),
        "learning_rate": raw_params[2],
        "train_batch_size": eval(raw_params[3]),
        "dropout": eval(raw_params[4].replace("-", ".")),
        "version": raw_params[5]   
    }

    if len(raw_params) > 6:
        params["search_max_size"] = eval(raw_params[6])
        params["filter_domain"] = eval(raw_params[7])
        params["filter_social_media"] = eval(raw_params[8])

    for idx, char in enumerate(params["learning_rate"]):
        if char == "-" and params["learning_rate"][idx-1] != "e":
            params["learning_rate"] = \
                params["learning_rate"][:idx] + "." + params["learning_rate"][idx+1:]

    params["learning_rate"] = eval(params["learning_rate"])
    params["experiment_name"] = experiment_name

    params = pd.Series(params)

    with open(os.path.join(experiment_path, "eval_results.txt")) as f:
        eval_metrics = f.read()
        eval_metrics = eval_metrics.strip().split("\n")

    eval_metrics = [metric.split(" = ") for metric in eval_metrics]
    eval_metrics = {metric[0]: eval(metric[1]) for metric in eval_metrics}
    eval_metrics = pd.Series(eval_metrics)
    eval_metrics.rename(
        {
            "mcc": "test_mcc",
            "accuracy": "test_acc",
            "f1_score": "test_f1",
            "tp": "test_tp",
            "tn": "test_tn",
            "fp": "test_fp",
            "fn": "test_fn",
            "auroc": "test_auroc",
            "auprc": "test_auprc",
            "eval_loss": "test_loss"
        },
        inplace=True
    )


    train_metrics = pd.read_csv(os.path.join(experiment_path, "training_progress_scores.csv"))
    train_metrics.rename(columns={
            "mcc": "dev_mcc",
            "accuracy": "dev_acc",
            "f1_score": "dev_f1",
            "tp": "dev_tp",
            "tn": "dev_tn",
            "fp": "dev_fp",
            "fn": "dev_fn",
            "auroc": "dev_auroc",
            "auprc": "dev_auprc",
            "global_step": "train_step",
            "eval_loss": "dev_loss"
        },
        inplace=True
    )

    train_metrics = train_metrics[train_metrics["dev_f1"] == train_metrics["dev_f1"].max()]
    train_metrics = train_metrics.iloc[-1]
    train_metrics[["dev_tp", "dev_tn", "dev_fp", "dev_fn"]] \
         = train_metrics[["dev_tp", "dev_tn", "dev_fp", "dev_fn"]].astype(int)
    

    experiment = pd.concat(
        [params, train_metrics, eval_metrics]
    )

    experiments.append(experiment)

experiments = pd.DataFrame(experiments)


  0%|          | 0/613 [00:00<?, ?it/s]

In [33]:
results = experiments.fillna(False).groupby(["data", "dataset", "filter_domain"])[["test_acc", "test_f1"]]
mean = (results.mean()*100).round(2)
std = (results.std()*100).round(2)
fmt = mean.astype(str) + " +- " +  std.astype(str)
fmt = fmt.reset_index()
fmt = fmt[fmt["data"] != "raw"]
fmt = fmt.set_index(["dataset", "data", "filter_domain"]).sort_index()

In [34]:
fmt

test_acc        test_f1
dataset    data            filter_domain                              
COVID19.BR processed       False           79.48 +- 2.9   78.9 +- 3.17
           processed_extra False          78.85 +- 6.03  78.09 +- 6.95
                           True           77.84 +- 6.55   77.0 +- 7.57
Fake.br    processed       False          97.86 +- 1.87  97.86 +- 1.87
           processed_extra False          97.52 +- 2.24  97.52 +- 2.25
                           True           97.47 +- 2.66  97.47 +- 2.69

In [37]:
llm_experiments = list()
analysis_llm = dict()

llm_v2_dir= "../output/llm_camera_ready"

for experiment_name in tqdm(os.listdir(llm_v2_dir)):
    experiment_path = os.path.join(llm_v2_dir, experiment_name)

    if not os.path.exists(os.path.join(experiment_path, "result.json")):
        continue

    raw_params = experiment_name.split("_")

    if "gemini-gemini" in raw_params[0]:
        raw_params[0] = raw_params[0].replace("gemini-gemini", "gemini/gemini")
    elif raw_params[:2] == ['vertex', 'ai-gemini-2-0-flash-001']:
        raw_params[0] = 'vertex_ai/gemini-2-0-flash-001'
        raw_params.remove('ai-gemini-2-0-flash-001')

    if "extra" in raw_params[2]:
        raw_params[1] = "processed_extra"
        raw_params.remove("extra")

    if len(raw_params) > 5:
        raw_params[5] = "_".join(raw_params[5:])
        raw_params = raw_params[:6]

    params = {
        "llm": raw_params[0],
        "data": raw_params[1],
        "dataset": raw_params[2].replace("-", "."),
        "few_shot": raw_params[3],
        "seed": raw_params[4],
        "mode": raw_params[5],
    }

    with open(os.path.join(experiment_path, "result.json")) as f:
        report = json.load(f)

    preds = pd.read_json(os.path.join(experiment_path, "requests_results.jsonl"), lines=True)

    params["test_acc"] = report["accuracy"]
    params["test_f1"] = report["classification_report"]["macro avg"]["f1-score"]
    params["test_tp"] = report["confusion_matrix"][0][0]
    params["test_tn"] = report["confusion_matrix"][1][1]
    params["test_fp"] = report["confusion_matrix"][0][1]
    params["test_fn"] = report["confusion_matrix"][1][0]

    llm_experiments.append(params)

    if params["dataset"] not in analysis_llm:
        analysis_llm[params["dataset"]] = preds

        analysis_llm[params["dataset"]]["text_a"] = analysis_llm[params["dataset"]]["dataset_row"].apply(lambda d: d.get("text_a", d.get("text")))


        if "text_b" in analysis_llm[params["dataset"]]["dataset_row"][0].keys():
            analysis_llm[params["dataset"]]["text_b"] = analysis_llm[params["dataset"]]["dataset_row"].apply(lambda d: d["text_b"])

        analysis_llm[params["dataset"]]["labels"] = analysis_llm[params["dataset"]]["dataset_row"].apply(lambda d: d["labels"])

        name = params["data"]

        analysis_llm[params["dataset"]].rename(
            columns={"is_correct": name}, inplace=True)

        #analysis_llm[params["dataset"]] = analysis_llm[params["dataset"]].set_index("dataset_index")

        analysis_llm[params["dataset"]].drop(
            columns=[
                "response_text","order", "dataset_row_tag_label", 
                "response_tag_label", "config", "response_raw", "response_label", 
                "model", "dataset_row"
            ],
            inplace=True
        )
    else:
        if "text_b" not in analysis_llm[params["dataset"]].columns and "text_b" in preds["dataset_row"][0]:
            analysis_llm[params["dataset"]]["text_b"] = preds["dataset_row"].apply(lambda d: d["text_b"])
        
        name = params["data"]
        analysis_llm[params["dataset"]][name] = preds["is_correct"]

llm_experiments = pd.DataFrame(llm_experiments)

  0%|          | 0/24 [00:00<?, ?it/s]

In [40]:
llm_experiments.sort_values(["data", "dataset", "mode"])

,llm,data,dataset,few_shot,seed,mode,test_acc,test_f1,test_tp,test_tn,test_fp,test_fn
0,gemini/gemini-1-5-flash,processed,COVID19.BR,15,1,no_social_media,0.765517,0.765517,111,111,55,13
2,gemini/gemini-1-5-flash,processed,COVID19.BR,15,2,no_social_media,0.765517,0.765517,111,111,55,13
4,gemini/gemini-1-5-flash,processed,COVID19.BR,15,3,no_social_media,0.765517,0.765517,111,111,55,13
1,gemini/gemini-1-5-flash,processed,COVID19.BR,15,1,social_media,0.765517,0.765517,111,111,55,13
3,gemini/gemini-1-5-flash,processed,COVID19.BR,15,2,social_media,0.765517,0.765517,111,111,55,13
5,gemini/gemini-1-5-flash,processed,COVID19.BR,15,3,social_media,0.765517,0.765517,111,111,55,13
18,gemini/gemini-1-5-flash,processed,Fake.br,15,1,no_social_media,0.813889,0.808387,354,232,6,128
20,gemini/gemini-1-5-flash,processed,Fake.br,15,2,no_social_media,0.813889,0.808387,354,232,6,128
22,gemini/gemini-1-5-flash,processed,Fake.br,15,3,no_social_media,0.813889,0.808387,354,232,6,128
19,gemini/gemini-1-5-flash,processed,Fake.br,15,1,social_media,0.813889,0.808387,354,232,6,128


In [3]:
extra = experiments[experiments["data"] == "processed_extra"]

extra = extra[(extra["search_max_size"] == 1) & (~extra["filter_domain"])].drop(columns="search_max_size")

dev_max_extra = extra.groupby(
    ["dataset", "data", "filter_social_media"]
)
dev_max_extra = dev_max_extra["dev_f1"].max()

best_dev_extra = extra[
    extra.apply(lambda row: row["dev_f1"] == dev_max_extra.loc[
        (row["dataset"], row["data"],  row["filter_social_media"])
    ], axis=True)
]
best_dev_extra = best_dev_extra.drop_duplicates(
    subset=["dataset", "data", "filter_social_media"]
)

train_max_extra = best_dev_extra.groupby(
    ["dataset", "data", "filter_social_media"]
)
train_max_extra = train_max_extra[["test_f1", "test_acc"]].max()
train_max_extra = (train_max_extra*100).round(1)
train_max_extra

test_f1  test_acc
dataset    data            filter_social_media                   
COVID19.BR processed_extra False                   82.1      82.4
                           True                    77.9      78.3
Fake.br    processed_extra False                   99.2      99.2
                           True                    98.7      98.8

In [4]:
no_extra = experiments[experiments["data"] != "processed_extra"]

dev_max_group = no_extra.groupby(["dataset", "data"])["dev_f1"].max()

best_dev = no_extra[
    no_extra.apply(lambda row: row["dev_f1"] == dev_max_group.loc[(row["dataset"], row["data"])], axis=True)
]
best_dev = best_dev.drop_duplicates(subset=["data", "dataset"])

train_max_group = best_dev.groupby(["dataset", "data"])[["test_f1", "test_acc"]].max()
train_max_group = (train_max_group*100).round(1)
train_max_group

test_f1  test_acc
dataset    data                        
COVID19.BR processed     81.1      81.4
           raw           81.9      82.1
Fake.br    processed     98.9      98.9
           raw           99.6      99.6

In [ ]:
results = pd.concat([train_max_extra.reset_index(), train_max_group.reset_index()])
result = results.fillna(False)

results["data"] = results.apply(
    lambda row: "filter_social_media" if row["filter_social_media"] == True else row["data"], axis=True
)
results["data"] = results["data"].map(
    {
        "raw": "1. Original",
        "processed": "2. Validado",
        "processed_extra": "3.a) Validado e Enquecido completo",
        "filter_social_media": "3.b) Validado e Enquecido filtrado"
    }
)

results.drop(columns="filter_social_media", inplace=True)

results.rename(columns={
    "dataset": "Conjunto de dados",
    "data": "Processamento",
    "test_f1": "F1 macro",
    "test_acc": "Acurácia"
}, inplace=True)

#results = results.pivot(
#    columns="Conjunto de dados", index="Processamento", values=["F1 macro", "Acurácia"]
#)

results





#.sort_values("test_f1")

In [6]:
experiments = [best_dev, best_dev_extra]
experiments = pd.concat(experiments)
df_analysis = {
    d: pd.read_parquet(f"../data/parquet/split/{d}.parquet") for d in ["Fake.br", "COVID19.BR"]
}


df_analysis = {d: 
    df_analysis[d][df_analysis[d]["new_split"] == "test"][["text_no_url", "label", "google_search_results"]] 
    for d in df_analysis}

In [7]:
for _, experiment in experiments.iterrows():
    if experiment["dataset"] == "COVID19.BR":
        continue

    analysis = pd.read_table(
        os.path.join("..", "output", "a100", experiment["experiment_name"], "test.tsv"), index_col=0
    )
    analysis.index = [str(idx) for idx in analysis.index]
    name = experiment["data"]

    if experiment["filter_social_media"] == True:
        name += "_filter_social_media"

    df_analysis[experiment["dataset"]][name] = analysis["pred_correct"]

In [8]:
llm_experiments = list()
analysis_llm = dict()

for experiment_name in tqdm(os.listdir(LLM_OUTPUT_DIR)):
    experiment_path = os.path.join(LLM_OUTPUT_DIR, experiment_name)

    if not os.path.exists(os.path.join(experiment_path, "result.json")):
        continue

    raw_params = experiment_name.split("_")

    if "gemini-gemini" in raw_params[0]:
        raw_params[0] = raw_params[0].replace("gemini-gemini", "gemini/gemini")
    elif raw_params[:2] == ['vertex', 'ai-gemini-2-0-flash-001']:
        raw_params[0] = 'vertex_ai/gemini-2-0-flash-001'
        raw_params.remove('ai-gemini-2-0-flash-001')

    if "extra" in raw_params[2]:
        raw_params[1] = "processed_extra"
        raw_params.remove("extra")

    link = raw_params[4] == "link"

    if link:
        raw_params.remove("link")

    if len(raw_params) > 5:
        raw_params[5] = "_".join(raw_params[5:])
        raw_params = raw_params[:6]

    params = {
        "llm": raw_params[0],
        "data": raw_params[1],
        "dataset": raw_params[2].replace("-", "."),
        "few_shot": raw_params[3],
        "version": raw_params[4],
        "search_link": link,
        "search_filter": raw_params[5] if len(raw_params) > 5 else None   
    }

    if params["version"] != "v4" and params["data"] == "processed_extra":
        continue

    if params["llm"] != "gemini/gemini-1-5-flash":
        continue

    if params["search_filter"] in ["filter_social_media_2", "filter_domains"]:
        continue

    with open(os.path.join(experiment_path, "result.json")) as f:
        report = json.load(f)

    preds = pd.read_json(os.path.join(experiment_path, "requests_results.jsonl"), lines=True)

    params["test_acc"] = report["accuracy"]
    params["test_f1"] = report["classification_report"]["macro avg"]["f1-score"]
    params["test_tp"] = report["confusion_matrix"][0][0]
    params["test_tn"] = report["confusion_matrix"][1][1]
    params["test_fp"] = report["confusion_matrix"][0][1]
    params["test_fn"] = report["confusion_matrix"][1][0]

    llm_experiments.append(params)

    if params["dataset"] not in analysis_llm:
        analysis_llm[params["dataset"]] = preds

        analysis_llm[params["dataset"]]["text_a"] = analysis_llm[params["dataset"]]["dataset_row"].apply(lambda d: d.get("text_a", d.get("text")))


        if "text_b" in analysis_llm[params["dataset"]]["dataset_row"][0].keys():
            analysis_llm[params["dataset"]]["text_b"] = analysis_llm[params["dataset"]]["dataset_row"].apply(lambda d: d["text_b"])

        analysis_llm[params["dataset"]]["labels"] = analysis_llm[params["dataset"]]["dataset_row"].apply(lambda d: d["labels"])

        name = params["data"]
        
        if params["search_filter"]:
            name += f"_{params['search_filter']}"

        analysis_llm[params["dataset"]].rename(
            columns={"is_correct": name}, inplace=True)

        #analysis_llm[params["dataset"]] = analysis_llm[params["dataset"]].set_index("dataset_index")

        analysis_llm[params["dataset"]].drop(
            columns=[
                "response_text","order", "dataset_row_tag_label", 
                "response_tag_label", "config", "response_raw", "response_label", 
                "model", "dataset_row"
            ],
            inplace=True
        )
    else:
        if "text_b" not in analysis_llm[params["dataset"]].columns and "text_b" in preds["dataset_row"][0]:
            analysis_llm[params["dataset"]]["text_b"] = preds["dataset_row"].apply(lambda d: d["text_b"])
        
        name = params["data"]
        if params["search_filter"]:
            name += f"_{params['search_filter']}"

        analysis_llm[params["dataset"]][name] = preds["is_correct"]

llm_experiments = pd.DataFrame(llm_experiments)
llm_experiments["filter_social_media"] = llm_experiments["search_filter"].map(lambda s: pd.notna(s))
llm_experiments.drop(columns=["search_link", "version", "few_shot", "llm", "search_filter"], inplace=True)
llm_experiments["data"] = llm_experiments.apply(
    lambda row: "filter_social_media" if row["filter_social_media"] == True else row["data"], axis=True
)

  0%|          | 0/34 [00:00<?, ?it/s]

In [9]:
llm_experiments["data"] = llm_experiments["data"].map(
    {
        "raw": "1. Original",
        "processed": "2. Validado",
        "processed_extra": "3.a) Validado e Enquecido completo",
        "filter_social_media": "3.b) Validado e Enquecido filtrado"
    }
)

llm_experiments.drop(columns="filter_social_media", inplace=True)

llm_experiments.rename(columns={
    "dataset": "Conjunto de dados",
    "data": "Processamento",
    "test_f1": "F1 macro",
    "test_acc": "Acurácia"
}, inplace=True)

llm_experiments = llm_experiments.set_index(
    ["Conjunto de dados", "Processamento"]
)
llm_experiments = llm_experiments.sort_index()

llm_experiments[["Acurácia", "F1 macro"]] = (100*llm_experiments[["Acurácia", "F1 macro"]]).round(1)

In [10]:
llm_experiments

Acurácia  F1 macro  \
Conjunto de dados Processamento                                            
COVID19.BR        1. Original                             75.2      75.2   
                  2. Validado                             76.9      76.9   
                  3.a) Validado e Enquecido completo      79.3      79.3   
                  3.b) Validado e Enquecido filtrado      79.0      78.9   
Fake.br           1. Original                             81.5      81.0   
                  2. Validado                             81.0      80.4   
                  3.a) Validado e Enquecido completo      77.6      76.7   
                  3.b) Validado e Enquecido filtrado      78.1      77.2   
MuMiN.PT          1. Original                             78.0      53.9   
                  2. Validado                             78.0      55.8   
                  3.a) Validado e Enquecido completo      70.9      50.9   
                  3.b) Validado e Enquecido filtrado      70.9      50.9   

                                                      test_tp  test_tn  \
Conjunto de dados Processamento                                          
COVID19.BR        1. Original                             108      110   
                  2. Validado                             111      112   
                  3.a) Validado e Enquecido completo      121      109   
                  3.b) Validado e Enquecido filtrado      122      107   
Fake.br           1. Original                             352      235   
                  2. Validado                             354      229   
                  3.a) Validado e Enquecido completo      351      208   
                  3.b) Validado e Enquecido filtrado      351      211   
MuMiN.PT          1. Original                               4      106   
                  2. Validado                               5      105   
                  3.a) Validado e Enquecido completo        5       95   
                  3.b) Validado e Enquecido filtrado        5       95   

                                                      test_fp  test_fn  
Conjunto de dados Processamento                                         
COVID19.BR        1. Original                              58       14  
                  2. Validado                              55       12  
                  3.a) Validado e Enquecido completo       45       15  
                  3.b) Validado e Enquecido filtrado       44       17  
Fake.br           1. Original                               8      125  
                  2. Validado                               6      131  
                  3.a) Validado e Enquecido completo        9      152  
                  3.b) Validado e Enquecido filtrado        9      149  
MuMiN.PT          1. Original                               2       29  
                  2. Validado                               1       30  
                  3.a) Validado e Enquecido completo        1       40  
                  3.b) Validado e Enquecido filtrado        1       40

In [22]:
from pprint import pprint

df = "Fake.br"

examples = analysis_llm[df]["request"][0][:-1]

pprint(examples)

[{'content': 'Abaixo contém textos de mensagens e notícias em português, sua '
             'tarefa é classificar se o texto contém uma Fake News ou Não.\n'
             'Para axiliar na classificação também será também fornecido um '
             'contexto extra que corresponde a uma busca no google pelos '
             'termos do texto a ser classificado.\n'
             'Responda apenas com  as tags "FAKE NEWS" ou "VERDADEIRO".',
  'role': 'system'},
 {'content': '---Contexto Extra---\n'
             '- Resultado de busca 1:\n'
             '  Título: Drama do Corinthians no Brasileirão 2024: Risco de '
             'Rebaixamento ...\n'
             '  Link: '
             'https://www.dgabc.com.br/Mobile/Noticia/4168197/drama-do-corinthians-no-brasileirao-2024-risco-de-rebaixamento-e-desafios\n'
             '  Trechos: Oct 8, 2024 ... Na última rodada, o Corinthians '
             'recebeu o Internacional na Neo Química Arena em um jogo que '
             'poderia ter sido o ponto